In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from blockOpInf.utils import damping
from blockOpInf.dataset import read_dataset

In [ ]:
frequencies = np.array([60.3136, 239.7978, 303.7807, 575.1929])

In [ ]:
k0, k1 = 300, -1
name = "wing-445.6"
mach_number_list = [0.901, 0.957, 1.141]
dynamic_pressure_list = [50, 70, 90]

mach_number = mach_number_list[0]
# dynamic_pressure = dynamic_pressure_list[0]
dynamic_pressures_mpm = []
for mach_number in mach_number_list:
    dampings = []
    for dynamic_pressure in dynamic_pressure_list:
        path = f"/storage/LMproj/agard/viscous/m{mach_number:.3f}_q{dynamic_pressure:d}/unsteady"
        print(f"-- [Mach number, Dynamic pressure]\t[{mach_number:.3f}, {dynamic_pressure:d}] --")

        # Load structural data
        dataset_SD_FOM = read_dataset(f"{path:s}/{name:s}_aehist.h5")
        QsFOM_ = dataset_SD_FOM.Q
        tsFOM = dataset_SD_FOM.t

        # Compute damping
        dampings.append(damping(tsFOM[k0:k1], QsFOM_[0, k0:k1], threshold=0.001, rho=1000.0)[0])

    dynamic_pressure_flutter = np.interp(0.0, dampings, dynamic_pressure_list)
    dynamic_pressures_mpm.append(dynamic_pressure_flutter.astype(float))
print(dynamic_pressures_mpm)

In [ ]:
# Training set grid
mach_numbers_train = [0.901, 0.957, 1.141]
dynamic_pressures_train = [50, 70, 90]
m_train_grid, q_train_grid = np.meshgrid(mach_numbers_train, dynamic_pressures_train)

# Experimental flutter boundary
mach_numbers_yates1987 = [0.499, 0.678, 0.901, 0.957, 1.072, 1.141]
dynamic_pressures_yates1987 = [133.10, 115.70, 89.30, 61.20, 66.10, 105.30]

# Computational flutter boundary - other literature
dynamic_pressures_silva2014 = [137, 125, 95, 75, 86, 215]

# Computational flutter boundary - fun3d and matrix pencil method

In [ ]:
fig, ax = plt.subplots()
ax.scatter(m_train_grid, q_train_grid, label="Training set grid",
           marker=".", color="C2", facecolors="None")
ax.scatter(mach_numbers_yates1987, dynamic_pressures_yates1987, label="Experiment (Yates 1987)",
        color="black")
ax.scatter(mach_numbers_yates1987, dynamic_pressures_silva2014, label="FUN3D NS SA (Silva 2014)",
        )
ax.scatter(mach_numbers_train, dynamic_pressures_mpm, label="Matrix pencil method",
        color="C2")
ax.set_xlim([0.85, 1.20])
ax.set_xlabel("Mach Number (-)")
ax.set_ylabel("Dynamic Pressure (psf)")
ax.legend()
plt.show()